# Aprendizado de máquina com o scikit-learn

*
   * Tipos de aprendizado de máquina
* Divisão de teste de trem
* Use `sklearn` para criar um modelo de regressão linear
* Codificação One-Hot
* Pipelines
* Métricas de avaliação

Faremos referência aos [documentos do scikit-learn] (https://scikit-learn.org/stable/user_guide.html) e [pandas docs] (https://pandas.pydata.org/pandas-docs/stable/ index.html), quando relevante, e analisará dados do conjunto de dados COVID-19 dos Estados Unidos do New York Times em https://github.com/nytimes/covid-19-data

** Isenção de responsabilidade: a regressão linear não é o algoritmo mais adequado para este conjunto de dados, mas estamos usando-o para ilustrar como usar o scikit-learn **

-sandbox
## Types of Machine Learning
* Supervised Learning
  * Regression <img src="https://miro.medium.com/max/640/1*LEmBCYAttxS6uI6rEyPLMQ.png" style="height: 250px; padding: 10px"/>
  * Classification
    <img src="https://cdn3-www.dogtime.com/assets/uploads/2018/10/puppies-cover.jpg" style="height: 250px; padding: 10px"/>
    <img src="https://images.unsplash.com/photo-1529778873920-4da4926a72c2?ixlib=rb-1.2.1&w=1000&q=80" style="height: 250px; padding: 10px"/>
* Unsupervised Learning
<img src="https://www.iotforall.com/wp-content/uploads/2018/01/Screen-Shot-2018-01-17-at-8.10.14-PM.png" style="height: 250px; padding: 10px"/>
* Reinforcement Learning
<img src="https://brookewenig.com/img/ReinforcementLearning/Rl_agent.png" style="height: 250px; padding: 10px"/>

Hoje vamos começar de maneira simples e focar em um problema de aprendizado supervisionado (regressão). Aqui, usaremos um modelo de regressão linear para prever o número de mortes resultantes do COVID-19.

In [4]:
%fs ls databricks-datasets/COVID/covid-19-data/us-states.csv

In [5]:
import pandas as pd

df = pd.read_csv("/dbfs/databricks-datasets/COVID/covid-19-data/us-states.csv")
df.head()
df.date.max()

In [6]:
df.shape

## Relationship between Cases & Deaths

In [8]:
# To allow us to print out plots
%matplotlib inline

In [9]:
# Filter to 2020-05-01
df_05_01 = df[df["date"] == "2020-05-01"]

ax = df_05_01.plot(x="cases", y="deaths", kind="scatter", 
                   figsize=(12,8), s=100, title="Deaths vs Cases on 2020-05-01 - All States")

df_05_01[["cases", "deaths", "state"]].apply(lambda row: ax.text(*row), axis=1);

## New York & New Jersey are Outliers

In [11]:
# Filter to states that are NOT New York and NOT New Jersey
not_ny = df[(df["state"] != "New York") & (df["state"] != "New Jersey")]
not_ny.head()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2608852735053438> in <module> 
 1 # Filter to states that are NOT New York and NOT New Jersey 
 ----> 2 not_ny = df [ ( df [ "state" ] != "New York" ) & ( df [ "state" ] != "New Jersey" ) ] 
 3 not_ny . head ( ) 

 NameError : name 'df' is not defined

In [12]:
# Filter to 2020-05-01
not_ny_05_01 = not_ny[not_ny["date"] == "2020-05-01"]

ax = not_ny_05_01.plot(x="cases", y="deaths", kind="scatter", 
                   figsize=(12,8), s=50, title="Deaths vs Cases on 2020-05-01 - All States but NY and NJ")

not_ny_05_01[["cases", "deaths", "state"]].apply(lambda row: ax.text(*row), axis=1);

## New York verus California COVID-19 deaths comparison

In [14]:
df_ny_cali = df[(df["state"] == "New York") | (df["state"] == "California")]

# Let's pivot our df_ny_cali DataFrame so that we can plot deaths over time for both states
df_ny_cali_pivot = df_ny_cali.pivot(index='date', columns='state', values='deaths').fillna(0)
df_ny_cali_pivot

In [15]:
df_ny_cali_pivot.plot.line(title="Deaths 2020-01-25 to 2020-05-01 - CA and NY", figsize=(12,8))

## Train-Test Split

![](https://brookewenig.com/img/IntroML/trainTest.png)

182/5000
Como se trata de dados temporais, em vez de fazer uma divisão aleatória, usaremos os dados de 1º de março a 7 de abril para treinar nosso modelo e testá-lo prevendo valores de 8 a 14 de abril.

In [18]:
train_df = df[(df["date"] >= "2020-03-01") & (df["date"] <= "2020-04-07")]
test_df = df[df["date"] > "2020-04-07"]

X_train = train_df[["cases"]]
y_train = train_df["deaths"]

X_test = test_df[["cases"]]
y_test = test_df["deaths"]

## Linear Regression

* Goal: Find the line of best fit
$$\hat{y} = w_0 + w_1x$$

$$\{y} ≈ \hat{y} + ϵ$$
* *x*: feature
* *y*: label

![](https://miro.medium.com/max/640/1*LEmBCYAttxS6uI6rEyPLMQ.png)

Aqui, ajustaremos um modelo de regressão linear do scikit-learn.

In [21]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
print(f"num_deaths = {lr.intercept_:.4f} + {lr.coef_[0]:.4f}*cases")

Hmmm... if we have no cases, then there should be no deaths caused by COVID-19, so let's set the intercept to be 0.

In [23]:
lr = LinearRegression(fit_intercept=False).fit(X_train, y_train)
print(f"num_deaths = {lr.coef_[0]:.4f}*cases")

So this model is implying that there is a 2.9% mortality rate in our dataset. But we know that some states have higher mortality rates than others. Let's include the state as a feature!

## One-Hot Encoding
Como lidamos com recursos não numéricos, como o estado?

Uma ideia:
* Crie um recurso numérico único para representar um não numérico
* Recursos categóricos:
   * state = {'Nova York', 'Califórnia', 'Louisiana'}
   * 'Nova York' = 1, 'Califórnia' = 2, 'Louisiana' = 3
  
MAS isso implica que a Califórnia é 2x Nova York!

Melhor ideia:
* Crie um recurso "fictício" para cada categoria
* 'Nova York' => [1, 0, 0], 'Califórnia' => [0, 1, 0], 'Louisiana' => [0, 0, 1]

Essa técnica é conhecida como ["One Hot Encoding"] (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [26]:
from sklearn.preprocessing import OneHotEncoder

X_train = train_df[["cases", "state"]]
X_test = test_df[["cases", "state"]]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc.fit(X_train).transform(X_train)

Let's check the shape

In [28]:
enc.fit(X_train).transform(X_train).shape

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2608852735053455> in <module> 
 ----> 1 enc . fit ( X_train ) . transform ( X_train ) . shape

 NameError : name 'enc' is not defined

Yikes! It one-hot encoded the cases variable too

In [30]:
enc.categories_

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2608852735053457> in <module> 
 ----> 1 enc . categories_

 NameError : name 'enc' is not defined

We need the [column transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to only apply the one hot encoding to a single column.

In [32]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([("enc", enc, ["state"])], remainder="passthrough")
ct.fit_transform(X_train).shape

## Pipelines

Podemos encadear uma série de transformações de dados com um [pipeline] (https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Dessa forma, também garantimos que, quaisquer que sejam as operações que aplicamos ao nosso conjunto de treinamento, também aplicamos na mesma ordem ao nosso conjunto de testes.

In [34]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[("ct", ct), ("lr", lr)])
pipeline_model = pipeline.fit(X_train, y_train)

y_pred = pipeline_model.predict(X_test)

## Como estão os diferentes estados?

Você notará que, adicionando recursos adicionais, nosso coeficiente para o recurso de casos também mudou.

In [36]:
print(f"num_deaths = {pipeline_model.steps[1][1].coef_[-1]:.4f}*cases + state_coef")

In [37]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

categories = pipeline_model.steps[0][1].transformers[0][1].categories_[1]

pd.DataFrame(zip(categories, pipeline_model.steps[1][1].coef_[:-1]), columns=["State", "Coefficient"])

## Evaluation Metrics

![](https://brookewenig.com/img/IntroML/RMSE.png)

Let's compute the MSE and RMSE for our dataset using the [sklearn.metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html?highlight=mean_squared_error).

In [40]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"MSE is {mse:.1f}, RMSE is {rmse:.1f}")

## Visualize Predictions

In [42]:
pred = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(y_pred, columns=["predicted_deaths"])], axis=1)
pred

Voila! You have successfully built a machine learning pipeline using scikit-learn!

To keep exploring with scikit-learn, checkout the datasets at [UCI ML Repository](https://archive.ics.uci.edu/ml/index.php) and [Kaggle](https://www.kaggle.com/)!